In [437]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [438]:
df = pd.read_excel('/Users/anjiho/2023-08-15_딥러닝 이론 및 실습/VMI_data.xls')

In [439]:
df

Part Number  D일06~08(08)H 투입계획(발주) 수량  D일08~10(10)H 투입계획(발주) 수량  \
0          Part 0                         0                         0   
1          Part 1                         0                         0   
2          Part 2                       139                        60   
3          Part 3                        15                        11   
4          Part 4                        40                        27   
...           ...                       ...                       ...   
17359    Part 105                         0                         0   
17360    Part 112                         0                         0   
17361    Part 106                       141                        51   
17362    Part 107                       104                        33   
17363    Part 108                         0                         5   

       D일10~12(13)H 투입계획(발주) 수량  D일13~15(15)H 투입계획(발주) 수량  \
0                             0                         0   
1                             0                         0   
2                            80                        76   
3                            17                        13   
4                            29                        17   
...                         ...                       ...   
17359                         0                         0   
17360                         0                         0   
17361                        69                        55   
17362                        46                        46   
17363                         1                         0   

       D일15~17(18)H 투입계획(발주) 수량  D일18~20(21)H 투입계획(발주) 수량  \
0                             0                         0   
1                             0                         0   
2                            68                        81   
3                            12                        16   
4                            18                        29   
...                         ...                       ...   
17359                         0                         0   
17360                         0                         0   
17361                        48                         0   
17362                        40                         0   
17363                         0                         0   

       D일21~23(23)H 투입계획(발주) 수량  D일23~01(02)H 투입계획(발주) 수량  D일 02~04H 투입계획 수량  \
0                             0                         0                  0   
1                             0                         0                  0   
2                            68                        44                  0   
3                            12                         8                  0   
4                            18                         9                  0   
...                         ...                       ...                ...   
17359                         0                         0                  0   
17360                         0                         0                  0   
17361                         0                         0                  0   
17362                         0                         0                  0   
17363                         0                         0                  0   

       ...  D+23일 투입예정 수량  D+24일 투입예정 수량  D+25일 투입예정 수량  D+26일 투입예정 수량  \
0      ...              0              0              0              0   
1      ...              0              0              0              0   
2      ...              0              0              0              0   
3      ...              0              0              0              0   
4      ...              0              0              0              0   
...    ...            ...            ...            ...            ...   
17359  ...              0              0              0              0   
17360  ...              0              0              0              0   
17361  ...              0           

In [440]:
df_drop_col = df.drop('Part Number',axis = 1)

In [441]:
df_drop_col = df_drop_col.dropna()

In [442]:
df_drop_col['CRET_TIME'] = pd.to_datetime(df_drop_col['CRET_TIME'], format='%Y%m%d%H%M')

In [443]:
df_drop_col['CRET_TIME']

0       2021-09-13 18:30:00
1       2021-09-13 18:30:00
2       2021-09-13 18:30:00
3       2021-09-13 18:30:00
4       2021-09-13 18:30:00
                ...        
17359   2021-11-01 07:03:00
17360   2021-11-01 07:03:00
17361   2021-11-01 07:03:00
17362   2021-11-01 07:03:00
17363   2021-11-01 07:03:00
Name: CRET_TIME, Length: 17364, dtype: datetime64[ns]

In [444]:
import pandas as pd
from datetime import datetime

# 기준 시간 설정
reference_time = datetime(2021, 9, 13, 18, 30, 0)

# 각 데이터 포인트의 시간과 기준 시간 사이의 차이를 계산하여 분 단위로 변환한 값을 새로운 컬럼에 저장합니다.
df_drop_col['time_difference_minutes'] = (df_drop_col['CRET_TIME']- reference_time).dt.total_seconds() / 3600


In [445]:
df_drop_col = df_drop_col.drop('CRET_TIME',axis = 1)

In [446]:
df_drop_col

D일06~08(08)H 투입계획(발주) 수량  D일08~10(10)H 투입계획(발주) 수량  \
0                             0                         0   
1                             0                         0   
2                           139                        60   
3                            15                        11   
4                            40                        27   
...                         ...                       ...   
17359                         0                         0   
17360                         0                         0   
17361                       141                        51   
17362                       104                        33   
17363                         0                         5   

       D일10~12(13)H 투입계획(발주) 수량  D일13~15(15)H 투입계획(발주) 수량  \
0                             0                         0   
1                             0                         0   
2                            80                        76   
3                            17                        13   
4                            29                        17   
...                         ...                       ...   
17359                         0                         0   
17360                         0                         0   
17361                        69                        55   
17362                        46                        46   
17363                         1                         0   

       D일15~17(18)H 투입계획(발주) 수량  D일18~20(21)H 투입계획(발주) 수량  \
0                             0                         0   
1                             0                         0   
2                            68                        81   
3                            12                        16   
4                            18                        29   
...                         ...                       ...   
17359                         0                         0   
17360                         0                         0   
17361                        48                         0   
17362                        40                         0   
17363                         0                         0   

       D일21~23(23)H 투입계획(발주) 수량  D일23~01(02)H 투입계획(발주) 수량  D일 02~04H 투입계획 수량  \
0                             0                         0                  0   
1                             0                         0                  0   
2                            68                        44                  0   
3                            12                         8                  0   
4                            18                         9                  0   
...                         ...                       ...                ...   
17359                         0                         0                  0   
17360                         0                         0                  0   
17361                         0                         0                  0   
17362                         0                         0                  0   
17363                         0                         0                  0   

       D일 04~06H 투입계획 수량  ...  D+23일 투입예정 수량  D+24일 투입예정 수량  D+25일 투입예정 수량  \
0                      0  ...              0              0              0   
1                      0  ...              0              0              0   
2                      0  ...              0              0              0   
3                      0  ...              0              0              0   
4                      0  ...              0              0              0   
...                  ...  ...            ...            ...            ...   
17359                  0  ...              0              0              0   
17360                  0  ...              0              0              0   
17361                  0  ...              0              0              0   
17362                  0  ...              0              0        

In [447]:
X = df_drop_col.drop('time_difference_minutes',axis = 1).values.astype(np.float32)

In [448]:
y = df_drop_col['time_difference_minutes'].values.astype(np.float32)

In [449]:
from sklearn.model_selection import train_test_split

In [450]:
X_train , X_test ,y_train,y_test = train_test_split(X,y,
                                                    train_size = 0.8,
                                                    random_state = 100)

X_train.shape,X_test.shape

((13891, 82), (3473, 82))

In [451]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [452]:
BATCH_SIZE=1
NUM_FEATURES=X.shape[1]
LEARNING_RATE=0.01
EPOCHS=100

In [453]:
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train)

train_ds = TensorDataset(X_train, y_train)
train_dl = DataLoader(train_ds,
                      batch_size=BATCH_SIZE,
                      shuffle=True)

In [454]:
model_s = nn.Sequential(nn.Linear(NUM_FEATURES, 10, bias=True), # layer 1
                      nn.ReLU(),
                      nn.Linear(10, 10, bias=True), # layer 2
                      nn.ReLU(),
                      nn.Linear(10, 10, bias=True), # layer 3
                      nn.ReLU(),
                      nn.Linear(10, 1, bias=True)) # output leayr=1 회귀

In [455]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_s.parameters(), lr=LEARNING_RATE)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_s.to(device)

Sequential(
  (0): Linear(in_features=82, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=10, bias=True)
  (3): ReLU()
  (4): Linear(in_features=10, out_features=10, bias=True)
  (5): ReLU()
  (6): Linear(in_features=10, out_features=1, bias=True)
)

In [456]:
for epoch in range(EPOCHS+1):
    epoch_loss=0.0
    for X_batch, y_batch in train_dl:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        pred = model_s(X_batch)
        loss = criterion(pred, y_batch)
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
        epoch_loss += loss.item()
        
    if epoch % 10 == 0:
        print(f'Epoch {epoch}: Loss {epoch_loss/len(train_dl):.3f}') 

/Users/anjiho/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/anjiho/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0: Loss 566427876.057
Epoch 10: Loss 370353386.334
Epoch 20: Loss 369634493.842
Epoch 30: Loss 369727984.397
Epoch 40: Loss 369314396.196
Epoch 50: Loss 369401305.726
Epoch 60: Loss 368548182.337
Epoch 70: Loss 368999653.778
Epoch 80: Loss 369156952.780
Epoch 90: Loss 369151432.417
Epoch 100: Loss 368297832.794


In [457]:
X_test = torch.from_numpy(X_test)
X_test = X_test.to(device)

y_preds = model_s(X_test).detach().cpu().numpy()

In [458]:
print('*MAE:', mean_absolute_error(y_test, y_preds))
print('*RMSE:', np.sqrt(mean_squared_error(y_test, y_preds)))
print('*R_Square:', r2_score(y_test, y_preds))

*MAE: 16507.74
*RMSE: 19352.576
*R_Square: 0.021534150892024595
